In [1]:
import pandas as pd
import os

In [2]:
DIR = os.getcwd() + "/nfl_predictor"
df = pd.read_csv(DIR + "/nfl_games.csv", index_col=0)

In [3]:
df = df.sort_values("date")
df = df.reset_index(drop=True)
del df["index_opp"]

In [4]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/1127067056.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("team", group_keys=False).apply(add_target)


In [5]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/1618397414.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/161839

In [6]:
df["target"] = df["target"].astype(int,errors="ignore")

In [7]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [8]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [10]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [11]:
predictors = list(selected_columns[sfs.get_support()])

In [12]:
predictors

['cmp',
 'att',
 'rate',
 'tgt',
 'rec',
 'lng.2',
 'att_max',
 'int_max',
 'att.1_max',
 'yds.2_max',
 'td.2_max',
 'fl_max',
 'total',
 'int_opp',
 'sk_opp',
 'lng_opp',
 'att.1_opp',
 'yds.2_opp',
 'lng.1_opp',
 'fmb_opp',
 'fl_opp',
 'int_max_opp',
 'lng_max_opp',
 'lng.1_max_opp',
 'rec_max_opp',
 'yds.3_max_opp',
 'td.2_max_opp',
 'lng.2_max_opp',
 'fl_max_opp',
 'total_opp']

In [13]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
        
    return pd.concat(all_predictions)

In [14]:
predictions = backtest(df, rr, predictors)

In [15]:
predictions

,actual,prediction
1068,1,0
1069,0,1
1070,1,1
1071,0,1
1072,1,0
...,...,...
4375,1,1
4376,2,1
4377,2,1
4378,2,0


In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.5461956521739131

In [17]:
df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/3128703651.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


home
0.0    0.445205
1.0    0.550685
dtype: float64

In [18]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]


In [19]:
def find_team_averages(team):
    numeric_columns = team.select_dtypes(include='number')
    rolling = numeric_columns.rolling(8).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/2356551130.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [20]:
df_rolling

,cmp,att,yds,td,int,sk,yds.1,lng,rate,att.1,...,tgt_max_opp,rec_max_opp,yds.3_max_opp,td.2_max_opp,lng.2_max_opp,fmb_max_opp,fl_max_opp,total_opp,home_opp,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,0.486413,0.507692,0.455357,0.208333,0.104167,0.147727,0.121951,0.298828,0.307926,0.415816,...,0.37500,0.257353,0.213260,0.18750,0.275266,0.15625,0.06250,0.226786,0.500,2023.0
4376,0.383152,0.421154,0.427124,0.354167,0.062500,0.238636,0.211890,0.296875,0.307829,0.520408,...,0.50000,0.389706,0.309966,0.21875,0.416223,0.25000,0.15625,0.232143,0.375,2023.0
4377,0.505435,0.505769,0.506515,0.291667,0.083333,0.181818,0.184451,0.292969,0.295255,0.464286,...,0.50625,0.433824,0.459037,0.25000,0.478723,0.15625,0.06250,0.337500,0.500,2023.0
4378,0.434783,0.461538,0.512548,0.291667,0.145833,0.170455,0.125000,0.367188,0.354862,0.466837,...,0.40625,0.345588,0.258446,0.25000,0.348404,0.18750,0.06250,0.332143,0.375,2023.0


In [21]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [22]:
df

,cmp,att,yds,td,int,sk,yds.1,lng,rate,att.1,...,tgt_max_opp_10,rec_max_opp_10,yds.3_max_opp_10,td.2_max_opp_10,lng.2_max_opp_10,fmb_max_opp_10,fl_max_opp_10,total_opp_10,home_opp_10,season_10
0,0.369565,0.461538,0.361004,0.166667,0.166667,0.272727,0.219512,0.109375,0.179726,0.551020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.369565,0.353846,0.330116,0.166667,0.333333,0.181818,0.231707,0.164062,0.178691,0.489796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.521739,0.646154,0.424710,0.000000,0.000000,0.000000,0.000000,0.132812,0.179467,0.510204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.413043,0.476923,0.370656,0.333333,0.000000,0.090909,0.000000,0.218750,0.245927,0.408163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.239130,0.353846,0.353282,0.000000,0.166667,0.272727,0.268293,0.421875,0.142229,0.326531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,0.630435,0.553846,0.451737,0.166667,0.000000,0.181818,0.134146,0.218750,0.259891,0.551020,...,0.37500,0.257353,0.213260,0.18750,0.275266,0.15625,0.06250,0.226786,0.500,2023.0
4376,0.413043,0.523077,0.511583,0.166667,0.166667,0.363636,0.207317,0.390625,0.195242,0.224490,...,0.50000,0.389706,0.309966,0.21875,0.416223,0.25000,0.15625,0.232143,0.375,2023.0
4377,0.521739,0.584615,0.513514,0.166667,0.000000,0.181818,0.158537,0.164062,0.229635,0.489796,...,0.50625,0.433824,0.459037,0.25000,0.478723,0.15625,0.06250,0.337500,0.500,2023.0
4378,0.500000,0.553846,0.519305,0.333333,0.000000,0.090909,0.048780,0.320312,0.640290,0.530612,...,0.40625,0.345588,0.258446,0.25000,0.348404,0.18750,0.06250,0.332143,0.375,2023.0


In [23]:
df = df.dropna()

In [24]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/1688004556.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["home_next"] = add_col(df, "home")
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_32757/1688004556.py:6: D

In [25]:
df = df.copy()

In [26]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
                left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

In [27]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,oti,sdg,sdg,oti,2016-11-06
1,sdg,oti,oti,sdg,2016-11-06
2,den,rai,rai,den,2016-11-06
3,crd,sfo,sfo,crd,2016-11-13
4,rai,den,den,rai,2016-11-06
...,...,...,...,...,...
2493,sfo,det,det,sfo,2024-01-28
2494,kan,rav,rav,kan,2024-01-28
2495,det,sfo,sfo,det,2024-01-28
2496,sfo,kan,kan,sfo,2024-02-11


In [28]:
df

,cmp,att,yds,td,int,sk,yds.1,lng,rate,att.1,...,td.2_max_opp_10,lng.2_max_opp_10,fmb_max_opp_10,fl_max_opp_10,total_opp_10,home_opp_10,season_10,home_next,team_opp_next,date_next
215,0.391304,0.307692,0.527027,0.333333,0.000000,0.000000,0.000000,0.414062,0.663046,0.775510,...,0.21875,0.389628,0.03125,0.03125,0.326786,0.375,2016.0,0.0,sdg,2016-11-06
216,0.413043,0.676923,0.501931,0.333333,0.500000,0.363636,0.256098,0.382812,0.126196,0.428571,...,0.21875,0.409574,0.15625,0.12500,0.378571,0.625,2016.0,1.0,oti,2016-11-06
217,0.413043,0.538462,0.519305,0.000000,0.166667,0.090909,0.109756,0.281250,0.168606,0.408163,...,0.15625,0.279255,0.21875,0.15625,0.242857,0.375,2016.0,0.0,rai,2016-11-06
219,0.565217,0.738462,0.652510,0.333333,0.333333,0.090909,0.073171,0.265625,0.386346,0.265306,...,0.37500,0.422872,0.12500,0.03125,0.425000,0.625,2016.0,1.0,dal,2016-11-06
220,0.739130,0.661538,0.687259,0.500000,0.166667,0.727273,0.573171,0.132812,0.287303,0.102041,...,0.15625,0.349734,0.34375,0.12500,0.250000,0.375,2016.0,1.0,sfo,2016-11-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,0.630435,0.553846,0.451737,0.166667,0.000000,0.181818,0.134146,0.218750,0.259891,0.551020,...,0.18750,0.275266,0.15625,0.06250,0.226786,0.500,2023.0,1.0,sfo,2024-02-11
4376,0.413043,0.523077,0.511583,0.166667,0.166667,0.363636,0.207317,0.390625,0.195242,0.224490,...,0.21875,0.416223,0.25000,0.15625,0.232143,0.375,2023.0,NaN,None,None
4377,0.521739,0.584615,0.513514,0.166667,0.000000,0.181818,0.158537,0.164062,0.229635,0.489796,...,0.25000,0.478723,0.15625,0.06250,0.337500,0.500,2023.0,NaN,None,None
4378,0.500000,0.553846,0.519305,0.333333,0.000000,0.090909,0.048780,0.320312,0.640290,0.530612,...,0.25000,0.348404,0.18750,0.06250,0.332143,0.375,2023.0,NaN,None,None


In [29]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [30]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [31]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [32]:
predictors = list(selected_columns[sfs.get_support()])

In [33]:
predictors

['rate',
 'lng_max',
 'lng.2_max',
 'fmb_max',
 'fl_max',
 'lng_opp',
 'yds.2_10_x',
 'lng.1_10_x',
 'yds.2_max_10_x',
 'td.1_max_10_x',
 'td.2_max_10_x',
 'total_10_x',
 'cmp_opp_10_x',
 'att_opp_10_x',
 'tgt_opp_10_x',
 'fl_opp_10_x',
 'cmp_max_opp_10_x',
 'att_max_opp_10_x',
 'yds.1_10_y',
 'att.1_max_10_y',
 'td.1_max_10_y',
 'td.2_max_10_y',
 'total_10_y',
 'lng_opp_10_y',
 'rate_opp_10_y',
 'cmp_max_opp_10_y',
 'att_max_opp_10_y',
 'lng_max_opp_10_y',
 'td.1_max_opp_10_y',
 'lng.1_max_opp_10_y']

In [34]:
predictions = backtest(full, rr, predictors)

In [35]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.6305263157894737